## This example demonstrates XCSF hyperparameter tuning

In [1]:
import json

import numpy as np
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_param_importances
from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from optuna.integration import OptunaSearchCV
from optuna.distributions import FloatDistribution

import xcsf

RANDOM_STATE: int = 1

np.random.seed(RANDOM_STATE)

### Load training and test data
Loads the kin8nm dataset from: https://www.openml.org/d/189

In [2]:
data = fetch_openml(data_id=189, as_frame=True, parser="auto")

data.frame.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
0,-0.015119,0.360741,0.469398,1.309675,0.988024,-0.025493,0.664071,0.062763,0.536524
1,0.360478,-0.301395,0.629183,-1.440146,-0.741637,-1.196749,-1.038444,-0.717461,0.308014
2,1.563238,-1.294753,0.078987,1.432937,1.149136,-1.292140,1.562988,-0.937731,0.518900
3,0.199485,0.901157,-1.356304,-0.080525,-0.976628,0.829894,-0.855649,0.930630,0.494151
4,0.659737,0.120552,-0.008756,0.648839,0.626832,-0.646539,1.318074,-0.899172,0.470218


In [3]:
data.frame.describe()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
count,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000
mean,0.009351,-0.007233,0.022792,-0.012945,0.007886,-0.009698,0.008402,0.001227,0.714283
std,0.904867,0.901496,0.900765,0.899186,0.906874,0.913243,0.909724,0.906098,0.263607
min,-1.570611,-1.570681,-1.570669,-1.570295,-1.570664,-1.570254,-1.570643,-1.568904,0.040165
25%,-0.770328,-0.778319,-0.749718,-0.780602,-0.770492,-0.802084,-0.776870,-0.777432,0.523584
50%,-0.002010,-0.015152,0.041346,-0.013617,0.009702,-0.003575,0.001338,0.002007,0.708822
75%,0.791179,0.779145,0.792391,0.760165,0.791203,0.781849,0.798014,0.780927,0.901354
max,1.570753,1.570657,1.570736,1.570169,1.570599,1.570597,1.570522,1.570237,1.458521


In [4]:
# numpy
X_orig = np.asarray(data.data, dtype=np.float64)
y_orig = np.asarray(data.target, dtype=np.float64)

# normalise inputs (zero mean and unit variance)
feature_scaler = StandardScaler()
X = feature_scaler.fit_transform(X_orig)

# reshape outputs into 2D arrays
if y_orig.ndim == 1:
    y_orig = y_orig.reshape(-1, 1)

# normalise outputs (zero mean and unit variance)
output_scaler = StandardScaler()
y = output_scaler.fit_transform(y_orig)

# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

# 10% of training for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=RANDOM_STATE
)

print(f"X_train shape = {np.shape(X_train)}")
print(f"y_train shape = {np.shape(y_train)}")
print(f"X_val shape = {np.shape(X_val)}")
print(f"y_val shape = {np.shape(y_val)}")
print(f"X_test shape = {np.shape(X_test)}")
print(f"y_test shape = {np.shape(y_test)}")

X_DIM: int = np.shape(X_train)[1]
Y_DIM: int = np.shape(y_train)[1]

X_train shape = (6634, 8)
y_train shape = (6634, 1)
X_val shape = (738, 8)
y_val shape = (738, 1)
X_test shape = (820, 8)
y_test shape = (820, 1)


### Initialise XCSF

In [5]:
# Due to the way XCSF sets parameters for conditions, actions, and predictions,
# it is necessary to set the desired parameters in a wrapper function...


def get_prediction(n_init: int) -> dict:
    """Returns parameters for initialising neural network predictions."""
    return {
        "type": "neural",
        "args": {
            "layer_0": {
                "type": "connected",
                "activation": "selu",
                "n_init": n_init,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
            "layer_1": {
                "type": "connected",
                "activation": "selu",
                "n_init": Y_DIM,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
        },
    }


def get_model() -> xcsf.XCS:
    """Returns a new XCS with baseline parameters."""
    return xcsf.XCS(
        x_dim=X_DIM,
        y_dim=Y_DIM,
        n_actions=1,
        omp_num_threads=12,
        random_state=RANDOM_STATE,
        pop_init=True,
        max_trials=200000,
        perf_trials=5000,
        pop_size=500,
        loss_func="mse",
        set_subsumption=False,
        theta_sub=100,
        e0=0.1,
        alpha=1,
        nu=20,
        beta=0.1,
        delta=0.1,
        theta_del=50,
        init_fitness=0.01,
        init_error=0,
        m_probation=10000,
        stateful=True,
        compaction=False,
        ea={
            "select_type": "roulette",
            "theta_ea": 50,
            "lambda": 2,
            "p_crossover": 0.8,
            "err_reduc": 1,
            "fit_reduc": 0.1,
            "subsumption": False,
            "pred_reset": False,
        },
        condition={
            "type": "tree_gp",
            "args": {
                "min_constant": 0,
                "max_constant": 1,
                "n_constants": 100,
                "init_depth": 5,
                "max_len": 10000,
            },
        },
        prediction=get_prediction(40),
    )

In [6]:
def get_callback():
    """Returns a new callback for early stoppping."""
    return xcsf.EarlyStoppingCallback(
        # note: PERF_TRIALS is considered an "epoch" for callbacks
        monitor="val",  # which metric to monitor: {"train", "val"}
        patience=20000,  # trials with no improvement after which training will be stopped
        restore_best=True,  # whether to make checkpoints and restore best population
        min_delta=0,  # minimum change to qualify as an improvement
        start_from=0,  # trials to wait before starting to monitor improvement
        verbose=True,  # whether to display when checkpoints are made
    )

### Sklearn pipeline

Note: sklearn pipelines do not allow the use of callback functions.

In [7]:
# split (original data) into training and test sets
X_tr, X_te, y_tr, y_te = train_test_split(
    X_orig, y_orig, test_size=0.1, random_state=RANDOM_STATE
)

model = get_model()
model.set_params(max_trials=50000)  # speed this demo up

pipe = make_pipeline(
    StandardScaler(),
    TransformedTargetRegressor(regressor=model, transformer=StandardScaler()),
)

pipe.fit(X_tr, y_tr)

00:00:00.643 trials=0 train=1.86722 pset=500.0 mset=238.9 mfrac=0.65
00:00:00.737 trials=5000 train=0.22114 pset=500.0 mset=223.5 mfrac=0.82
00:00:00.870 trials=10000 train=0.18990 pset=500.0 mset=366.9 mfrac=1.00
00:00:01.289 trials=15000 train=0.24111 pset=500.0 mset=405.6 mfrac=0.98
00:00:01.330 trials=20000 train=0.14901 pset=500.0 mset=378.3 mfrac=0.96
00:00:01.277 trials=25000 train=0.15173 pset=500.0 mset=414.2 mfrac=0.94
00:00:01.366 trials=30000 train=0.13972 pset=500.0 mset=406.4 mfrac=0.37
00:00:01.300 trials=35000 train=0.12362 pset=500.0 mset=394.7 mfrac=0.68
00:00:01.368 trials=40000 train=0.11709 pset=500.0 mset=390.1 mfrac=0.87
00:00:01.682 trials=45000 train=0.11693 pset=500.0 mset=384.0 mfrac=0.99
00:00:11.862


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=<xcsf.xcsf.XCS object at 0x7f6522d7c5f0>,
                                            transformer=StandardScaler()))])

In [8]:
train_score = pipe.score(X_tr, y_tr)
test_score = pipe.score(X_te, y_te)

print(f"train = {train_score}")
print(f"test = {test_score}")

train = 0.8870088321372127
test = 0.8812029081717271


### Sklearn grid search
General parameters can be searched in the usual way:

In [9]:
parameters = {"beta": [0.1, 0.5]}

model = get_model()
model.set_params(max_trials=25000)  # speed this demo up

grid_search = GridSearchCV(
    model,
    parameters,
    scoring="neg_mean_squared_error",
)

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

for mean_score, std_score, params in zip(
    results["mean_test_score"], results["std_test_score"], results["params"]
):
    print("Mean Score:", -mean_score)
    print("Standard Deviation:", std_score)
    print("Parameters:", params)
    print("------------------------")

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

00:00:00.829 trials=0 train=1.53248 pset=500.0 mset=253.6 mfrac=1.00
00:00:00.860 trials=5000 train=0.27342 pset=500.0 mset=244.3 mfrac=0.97
00:00:01.037 trials=10000 train=0.21938 pset=500.0 mset=428.5 mfrac=0.88
00:00:01.551 trials=15000 train=0.15557 pset=500.0 mset=453.9 mfrac=0.77
00:00:02.061 trials=20000 train=0.23632 pset=500.0 mset=444.7 mfrac=0.70
00:00:06.338
00:00:01.446 trials=0 train=1.31473 pset=500.0 mset=228.7 mfrac=0.94
00:00:00.806 trials=5000 train=0.49361 pset=500.0 mset=213.8 mfrac=0.88
00:00:01.020 trials=10000 train=0.21914 pset=500.0 mset=395.3 mfrac=0.72
00:00:01.407 trials=15000 train=0.14930 pset=500.0 mset=431.7 mfrac=0.87
00:00:01.446 trials=20000 train=0.13562 pset=500.0 mset=442.4 mfrac=0.95
00:00:06.125
00:00:00.773 trials=0 train=1.36022 pset=500.0 mset=239.1 mfrac=0.93
00:00:00.747 trials=5000 train=0.23166 pset=500.0 mset=222.0 mfrac=0.75
00:00:01.098 trials=10000 train=0.24337 pset=500.0 mset=398.8 mfrac=0.59
00:00:01.573 trials=15000 train=0.32053 

EA parameters require specifying a dict, but individual values can still be set because the other values are not changed.

In [10]:
parameters = {"ea": [{"lambda": 2}, {"lambda": 10}, {"lambda": 50}]}

However, for actions, conditions, and predictions, the WHOLE dict must be specified
for each value to try in the search. This is because of the way XCSF uses kwargs to
initialise values and they are reset each time. XCSF has so many different parameters
that it is unfortunately necessary to do it this way.

### Optuna

In [11]:
def objective(trial):
    """Measure a new hyperparameter combination."""
    # get new parameters to try
    e0: float = trial.suggest_float("e0", 0.08, 0.12, step=0.01)
    beta: float = trial.suggest_float("beta", 0.01, 0.19, step=0.02)
    n_init: int = trial.suggest_int("n_init", 1, 101, step=10)
    # get a new XCSF and set params
    prediction: dict = get_prediction(n_init=n_init)
    model = get_model()
    model.set_params(e0=e0, beta=beta, prediction=prediction)
    # get a new early stopping callback
    callback = get_callback()
    # fit XCSF
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=[callback],
        verbose=True,
    )
    # optimise validation MSE
    return model.score(X_val, y_val)


study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))

[I 2023-07-18 22:25:27,892] A new study created in memory with name: no-name-4ac6c624-2b60-438f-8120-e11ab0230c1c


In [12]:
study.optimize(objective, n_trials=10)

00:00:00.418 trials=0 train=0.66983 val=0.62794 pset=500.0 mset=277.0 mfrac=0.04
checkpoint: 0.62794 error at 0 trials
00:00:00.442 trials=5000 train=0.61953 val=0.66134 pset=500.0 mset=310.9 mfrac=0.31
00:00:00.577 trials=10000 train=0.59213 val=0.60278 pset=500.0 mset=409.6 mfrac=0.13
checkpoint: 0.60278 error at 10000 trials
00:00:00.616 trials=15000 train=0.57788 val=0.60331 pset=500.0 mset=400.8 mfrac=0.04
00:00:00.619 trials=20000 train=0.59493 val=0.59348 pset=500.0 mset=378.7 mfrac=0.45
checkpoint: 0.59348 error at 20000 trials
00:00:00.619 trials=25000 train=0.56567 val=0.61692 pset=500.0 mset=358.6 mfrac=1.00
00:00:00.602 trials=30000 train=0.56864 val=0.59377 pset=500.0 mset=345.8 mfrac=0.25
00:00:00.615 trials=35000 train=0.58743 val=0.70308 pset=500.0 mset=338.0 mfrac=0.77
00:00:00.609 trials=40000 train=0.57657 val=0.61365 pset=500.0 mset=321.6 mfrac=0.31


[I 2023-07-18 22:25:33,702] Trial 0 finished with value: 0.5934787014673956 and parameters: {'e0': 0.1, 'beta': 0.15000000000000002, 'n_init': 1}. Best is trial 0 with value: 0.5934787014673956.


00:00:00.598 trials=45000 train=0.55809 val=0.60627 pset=500.0 mset=314.2 mfrac=0.04
restoring system from trial 20000 with error=0.59348
00:00:05.715
00:00:00.692 trials=0 train=2.37700 val=0.29542 pset=500.0 mset=251.4 mfrac=1.00
checkpoint: 0.29542 error at 0 trials
00:00:00.660 trials=5000 train=0.23897 val=0.26776 pset=500.0 mset=277.0 mfrac=1.00
checkpoint: 0.26776 error at 5000 trials
00:00:00.895 trials=10000 train=0.22205 val=0.25417 pset=500.0 mset=467.0 mfrac=1.00
checkpoint: 0.25417 error at 10000 trials
00:00:01.100 trials=15000 train=0.20618 val=0.23715 pset=500.0 mset=482.0 mfrac=1.00
checkpoint: 0.23715 error at 15000 trials
00:00:01.134 trials=20000 train=0.17702 val=0.19186 pset=500.0 mset=487.0 mfrac=1.00
checkpoint: 0.19186 error at 20000 trials
00:00:01.154 trials=25000 train=0.17054 val=0.17992 pset=500.0 mset=491.0 mfrac=1.00
checkpoint: 0.17992 error at 25000 trials
00:00:01.162 trials=30000 train=0.17472 val=0.16862 pset=500.0 mset=491.1 mfrac=1.00
checkpoint: 

[I 2023-07-18 22:26:03,918] Trial 1 finished with value: 0.14756528276726605 and parameters: {'e0': 0.09, 'beta': 0.03, 'n_init': 11}. Best is trial 1 with value: 0.14756528276726605.


00:00:00.957 trials=135000 train=0.12032 val=0.14934 pset=500.0 mset=345.8 mfrac=1.00
restoring system from trial 110000 with error=0.14757
00:00:30.070
00:00:03.878 trials=0 train=6.62380 val=0.25126 pset=500.0 mset=256.2 mfrac=1.00
checkpoint: 0.25126 error at 0 trials
00:00:05.447 trials=5000 train=0.33433 val=0.23027 pset=500.0 mset=243.1 mfrac=0.50
checkpoint: 0.23027 error at 5000 trials
00:00:01.079 trials=10000 train=0.19652 val=0.19619 pset=500.0 mset=382.4 mfrac=1.00
checkpoint: 0.19619 error at 10000 trials
00:00:01.269 trials=15000 train=0.63862 val=0.18836 pset=500.0 mset=366.2 mfrac=0.13
checkpoint: 0.18836 error at 15000 trials
00:00:01.219 trials=20000 train=0.18317 val=0.19169 pset=500.0 mset=348.7 mfrac=0.25
00:00:01.235 trials=25000 train=0.19676 val=0.17326 pset=500.0 mset=353.3 mfrac=0.69
checkpoint: 0.17326 error at 25000 trials
00:00:01.274 trials=30000 train=0.18676 val=0.17340 pset=500.0 mset=362.8 mfrac=0.73
00:00:01.282 trials=35000 train=0.15162 val=0.16378 

...

00:00:02.305 trials=160000 train=0.08317 val=0.10560 pset=500.0 mset=409.9 mfrac=1.00
restoring system from trial 135000 with error=0.10420
00:01:11.577
00:00:02.104 trials=0 train=0.68861 val=0.69311 pset=500.0 mset=243.8 mfrac=0.41
checkpoint: 0.69311 error at 0 trials
00:00:01.458 trials=5000 train=0.62427 val=0.61960 pset=500.0 mset=259.0 mfrac=0.49
checkpoint: 0.61960 error at 5000 trials
00:00:00.599 trials=10000 train=0.58836 val=0.68150 pset=500.0 mset=399.9 mfrac=0.74
00:00:00.625 trials=15000 train=0.57764 val=0.63622 pset=500.0 mset=378.3 mfrac=0.21
00:00:00.604 trials=20000 train=0.57469 val=0.60491 pset=500.0 mset=340.8 mfrac=0.00
checkpoint: 0.60491 error at 20000 trials
00:00:00.624 trials=25000 train=0.55018 val=0.61081 pset=500.0 mset=319.8 mfrac=0.00
00:00:00.614 trials=30000 train=0.58060 val=0.60968 pset=500.0 mset=310.5 mfrac=0.00
00:00:00.624 trials=35000 train=0.57227 val=0.60938 pset=500.0 mset=298.8 mfrac=0.91
00:00:00.631 trials=40000 train=0.56762 val=0.62656

[I 2023-07-18 22:32:08,361] Trial 8 finished with value: 0.5834895718998512 and parameters: {'e0': 0.12, 'beta': 0.17, 'n_init': 1}. Best is trial 7 with value: 0.10419906184567795.


00:00:00.443 trials=95000 train=0.56258 val=0.64163 pset=500.0 mset=139.3 mfrac=0.00
restoring system from trial 70000 with error=0.58349
00:00:13.781
00:00:02.211 trials=0 train=10.75420 val=0.42921 pset=500.0 mset=275.0 mfrac=1.00
checkpoint: 0.42921 error at 0 trials
00:00:02.002 trials=5000 train=0.50441 val=0.26475 pset=500.0 mset=305.7 mfrac=1.00
checkpoint: 0.26475 error at 5000 trials
00:00:02.762 trials=10000 train=0.68747 val=1.02319 pset=500.0 mset=484.3 mfrac=1.00
00:00:03.525 trials=15000 train=0.35618 val=0.23635 pset=500.0 mset=486.5 mfrac=1.00
checkpoint: 0.23635 error at 15000 trials
00:00:03.106 trials=20000 train=0.33337 val=0.20877 pset=500.0 mset=487.7 mfrac=1.00
checkpoint: 0.20877 error at 20000 trials
00:00:03.086 trials=25000 train=0.27043 val=0.17449 pset=500.0 mset=490.0 mfrac=1.00
checkpoint: 0.17449 error at 25000 trials
00:00:03.127 trials=30000 train=0.34001 val=0.18166 pset=500.0 mset=495.0 mfrac=1.00
00:00:03.221 trials=35000 train=0.26608 val=0.17106 p

[I 2023-07-18 22:33:17,211] Trial 9 finished with value: 0.12359380958796687 and parameters: {'e0': 0.08, 'beta': 0.03, 'n_init': 91}. Best is trial 7 with value: 0.10419906184567795.


00:00:03.399 trials=105000 train=0.11087 val=0.12374 pset=500.0 mset=497.0 mfrac=1.00
restoring system from trial 80000 with error=0.12359
00:01:08.546


In [13]:
best_trial = study.best_trial
print(f"Best trial - Evaluation metric: {best_trial.value:.4f}")
print(f"Best trial - Hyperparameters: {best_trial.params}")

Best trial - Evaluation metric: 0.1042
Best trial - Hyperparameters: {'e0': 0.12, 'beta': 0.06999999999999999, 'n_init': 71}


In [ ]:
plot_param_importances(study)